# Analysis of the Professional Counter Strike Global Offensive Circuit
### CMSC 320 Final Project
### Richard Chen
___

## Introduction and Motivation
____
![](https://seeklogo.com/images/C/csgo-logo-CAA0A4D48A-seeklogo.com.png)


Counter-Strike Global Offensive (CS:GO) is a competitive PC game developed by Valve Corporation in 2012. It is part of a franchise of games stretching back to the release of the first Counter-Strike in 2000. CS:GO is played in a first person perspective, in which two teams of 5 players compete to see who wins a total of 16 rounds the fastest. For the first 15 rounds of the game, one side will play an offensive side (T-side) that is tasked with planting and bomb and having it detonate. The other side is tasked with defusing the bomb or preventing it from being planted at all (CT-side). Both teams can also win the round by eliminating all enemy players. In case both teams draw at the 30th round on 15-15, 6 more rounds are added as overtime; ends if a team wins 4 out of 6 rounds. At any point, there are 7 active maps in the competitive pool. Overall matches are played in a **best of 3 maps** to see who wins the series overall.

Because of the competitive nature of the game, a professional circuit has thrived over the past 8 years. With record player numbers in the base game (1.1 million concurrent players), and increased investment from traditional North American sports franchises, such as the Dallas Cowboys, CS:GO might be a sneak preview into the **future of entertainment**. Having a predictive model of how a competitive CS:GO team can be successful could be worth **millions** as the industry becomes more mainstream. Success in this model could open more opportunities for the broader sport of esports. 

In this tutorial, I will be analyzing a dataset from Kaggle Datasets. I will be going step-by-step through the data pipeline of professional CS:GO teams to determine winning trends and strategies. My goal is to lay the framework and an example for others to build upon when analyzing a competitive multiplayer game. This tutorial and analysis will be written with Python 3 in mind.

## Dataset

The dataset used for this tutorial analysis was found via Kaggle Datasets and can be found [here.](https://www.kaggle.com/mateusdmachado/csgo-professional-matches) Once you download the files from the link, you should find 4 csv files that stores the data we're interested in. This dataset includes data about competitive CS:GO matches between November 2015 to March 2020. The original source where the data was originally scraped from was www.hltv.org.

 - Results.csv: stores data about map scores and team rankings
 - Picks.csv: stores data about team's map picks and vetos during the prematch selection between opposing teams
 - Economy.csv: stores data about round start equipment values for all rounds played
 - Players.csv: stores individual performances of professional players on each map

## Getting Started

First we'll need to import the following python libraries:
 - [Pandas](https://pandas.pydata.org/)
 - [Beautiful Soup (OPTIONAL)](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
 
The syntax to install the libraries is "pip install < library name >"

In [54]:
# Importing the required libraries
import pandas as pd

## Data Scraping
Because the author of the CS:GO Kaggle dataset already scrapped the data off www.hltv.org, I will not be explicitly showing the steps needed to perform that. However, more information can be found by exploring documentation of **Beautiful Soup**, a Python library used to get data and parse a HTML page. [Link here to explore Beautiful Soup.](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## Data Tidying
After the data is scrapped, I use the **Pandas library's read_csv function** to load it into a a data structure called a "DataFrame." This will allow me to easily manipulate rows and columns based on their values. A DataFrame will be the basis of making our data usable for the rest of the pipeline. Essentially one row will represent one matchup between teams.

In [165]:
# loading data from our data into a dataframe
df = pd.read_csv("./csgo-professional-matches/results.csv")
# displaying the first few items of our datafram
df.head()

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
0,2020-03-18,Recon 5,TeamOne,Dust2,0,16,2,2,0,1,0,15,5151,2340454,62,63,0,2,2
1,2020-03-18,Recon 5,TeamOne,Inferno,13,16,2,2,8,6,5,10,5151,2340454,62,63,0,2,2
2,2020-03-18,New England Whalers,Station7,Inferno,12,16,2,1,9,6,3,10,5243,2340461,140,118,12,16,2
3,2020-03-18,Rugratz,Bad News Bears,Inferno,7,16,2,2,0,8,7,8,5151,2340453,61,38,0,2,2
4,2020-03-18,Rugratz,Bad News Bears,Vertigo,8,16,2,2,4,5,4,11,5151,2340453,61,38,0,2,2


At this point, we will use functions within the Pandas library to make our DataFrame more usable. One of which is to eliminate excessive data that we will not use in our analysis moving forward. For example in this CS:GO case, we will eliminate matches that did not feature top 20 teams. We will also drop columns that will not be helpful moving forward.

Functions used here:
 - **df = df.drop('column_name',1)** 
     - will be used to delete 'column_name' in a Pandas DataFrame. 1 is the axis number (0 for rows and 1 for columns).
 - **df = df.loc[df['column_name']...]** 
     - this will be used to filter the DataFrame based on the value of column_name. Could use any boolean statement of comparison in the "..." part 
 - **df['column_name'] = df.apply(lambda row: .... if ... else ...,axis =1)** 
     - this is the syntax used to create a new column based on already existing values of a particular row
 - **df.rename(index = str, columns = {"old_column_name": "new_column_name", ....}, inplace = True)** 
     - this can be used to rename columns and make it more presentable for the data pipeline further down
 - **df['date'] = pd.to_datetime(df['date'])** 
     - can be used to convert a column into a datetime object
 - **df = df_reset_index(drop = True)** 
     - can be used to reset the index

In [166]:
# cleaning up data
df = df.drop('starting_ct',1)
df = df.drop('result_1',1)
df = df.drop('result_2',1)
df = df.drop('map_wins_1',1)
df = df.drop('map_wins_2',1)
df = df.drop('event_id',1)
df = df.loc[df['rank_1'] <= 20]
df = df.loc[df['rank_2'] <= 20]

df['MapWinner'] = df.apply(lambda row: row.team_2 if row.map_winner == 2 else row.team_1, axis = 1)
df['SeriesWinner'] = df.apply(lambda row: row.team_2 if row.match_winner == 2 else row.team_1, axis = 1)
df['date'] = pd.to_datetime(df['date'])
df['Year'] = df.apply(lambda row: row.date.year, axis = 1)
df = df.drop('date',1)

df = df.drop('map_winner',1)
df = df.drop('match_winner',1)
df.rename(columns = {'_map':'Map','team_1':'Team_One','team_2':'Team_Two','rank_1':'Team_Ones_Rank','rank_2': 'Team_Twos_Rank', \
                    'ct_1': 'Team_One_CT_Rounds','t_2': 'Team_Two_T_Rounds', 't_1':'Team_One_T_Rounds', 'ct_2':'Team_Two_CT_Rounds'}, inplace = True)
df = df.sort_values(by = 'match_id', ascending = False)
df = df.reset_index(drop = True)
df

,Team_One,Team_Two,Map,Team_One_CT_Rounds,Team_Two_T_Rounds,Team_One_T_Rounds,Team_Two_CT_Rounds,match_id,Team_Ones_Rank,Team_Twos_Rank,MapWinner,SeriesWinner,Year
0,fnatic,BIG,Mirage,9,6,3,10,2340397,5,18,BIG,BIG,2020
1,fnatic,BIG,Inferno,5,10,2,6,2340397,5,18,BIG,BIG,2020
2,fnatic,BIG,Overpass,4,4,12,3,2340397,5,18,fnatic,BIG,2020
3,GODSENT,OG,Train,12,3,4,0,2340183,18,16,GODSENT,GODSENT,2020
4,GODSENT,OG,Dust2,9,6,6,9,2340183,18,16,OG,GODSENT,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5768,Liquid,Luminosity,Cobblestone,9,6,7,0,2299042,14,9,Liquid,Liquid,2015
5769,CLG,Luminosity,Cache,1,4,3,12,2299040,10,9,Luminosity,Luminosity,2015
5770,CLG,Liquid,Inferno,7,8,9,4,2299011,10,14,CLG,CLG,2015
5771,NiP,Envy,Cobblestone,4,6,12,3,2299003,6,1,NiP,NiP,2015


## Entity Resolution
Potential issue that might arise when using multiple datasets; for example, if the data scientist wanted to use data from Players.csv. I will now show an example of **joining** two DataFrames together. If the question was: "I want to see how this individual player performs when his team wins vs how this player performs when his team loses" In order to analyze such a question, you would join the data on match_id

In [189]:
player_df = pd.read_csv("./csgo-professional-matches/players.csv")
player_df = player_df[['player_name','team', 'match_id', 'm1_rating_ct', 'm1_rating_t', 'm2_rating_ct', \
                       'm2_rating_t','m3_rating_ct', 'm3_rating_t']]
player_df = player_df.dropna(subset=['m1_rating_ct', 'm1_rating_t', 'm2_rating_ct','m2_rating_t'])
player_df['Average_T_Rating'] = player_df[['m1_rating_t', 'm2_rating_t', 'm3_rating_t']].mean(axis=1)
player_df['Average_CT_Rating'] = player_df[['m1_rating_ct', 'm2_rating_ct', 'm3_rating_ct']].mean(axis=1)
player_df = player_df.drop(['m1_rating_ct','m1_rating_t', 'm2_rating_ct','m2_rating_t','m3_rating_ct',\
                            'm3_rating_t'],1)
player_df

,player_name,team,match_id,Average_T_Rating,Average_CT_Rating
0,Brehze,Evil Geniuses,2339385,0.860000,1.080000
1,CeRq,Evil Geniuses,2339385,0.676667,1.150000
2,EliGE,Liquid,2339385,0.876667,1.486667
3,Ethan,Evil Geniuses,2339385,0.413333,1.096667
4,NAF,Liquid,2339385,0.876667,1.273333
...,...,...,...,...,...
367897,Pimp,SK,2300263,0.730000,0.550000
367898,Spiidi,mousesports,2300263,1.275000,1.375000
367899,chrisJ,mousesports,2300263,1.160000,1.315000
367900,denis,mousesports,2300263,1.635000,1.505000


In [190]:
new_df = pd.merge(player_df,df,on = 'match_id')
new_df = new_df.sort_values(by = 'match_id', ascending = False)
new_df = new_df.reset_index(drop = True)
new_df

,player_name,team,match_id,Average_T_Rating,Average_CT_Rating,Team_One,Team_Two,Map,Team_One_CT_Rounds,Team_Two_T_Rounds,Team_One_T_Rounds,Team_Two_CT_Rounds,Team_Ones_Rank,Team_Twos_Rank,MapWinner,SeriesWinner,Year
0,ZywOo,Vitality,2339391,1.203333,1.650000,Vitality,FaZe,Nuke,5,7,6,9,7,9,FaZe,FaZe,2020
1,NiKo,FaZe,2339391,1.046667,1.376667,Vitality,FaZe,Inferno,5,10,5,6,7,9,FaZe,FaZe,2020
2,olofmeister,FaZe,2339391,0.976667,1.100000,Vitality,FaZe,Nuke,5,7,6,9,7,9,FaZe,FaZe,2020
3,olofmeister,FaZe,2339391,0.976667,1.100000,Vitality,FaZe,Mirage,7,8,9,6,7,9,Vitality,FaZe,2020
4,coldzera,FaZe,2339391,1.033333,1.256667,Vitality,FaZe,Inferno,5,10,5,6,7,9,FaZe,FaZe,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32635,Friis,SK,2300263,0.450000,0.390000,SK,mousesports,Mirage,3,12,5,4,18,12,mousesports,mousesports,2016
32636,Friis,SK,2300263,0.450000,0.390000,SK,mousesports,Cache,4,11,0,5,18,12,mousesports,mousesports,2016
32637,AcilioN,SK,2300263,0.875000,0.630000,SK,mousesports,Mirage,3,12,5,4,18,12,mousesports,mousesports,2016
32638,AcilioN,SK,2300263,0.875000,0.630000,SK,mousesports,Cache,4,11,0,5,18,12,mousesports,mousesports,2016


## Sources

- [Kaggle CS:GO Professional Matches](https://www.kaggle.com/mateusdmachado/csgo-professional-matches)
- [Pandas Python Library](https://pandas.pydata.org/)
- [Beautiful Soup]([https://www.crummy.com/software/BeautifulSoup/bs4/doc/](https://www.crummy.com/software/BeautifulSoup/bs4/doc/))
